In [ ]:
import pandas as pd
from sklearn import svm

In [ ]:
data_raw = pd.read_csv('data_train.csv')

In [ ]:
data_raw.head()

In [ ]:
data = data_raw
data['User'] = [(ID.split('_')[0])[1:] for ID in data_raw['Id']]
data['Movie'] = [(ID.split('_')[1])[1:] for ID in data_raw['Id']]

In [ ]:
data.head()

In [ ]:
X = data[['User', 'Movie', 'Prediction']]
print(X.shape)

Y = data['Prediction']
print(Y.shape)


In [ ]:
import findspark
#findspark.init('/srv/spark')
findspark.init()
import pyspark
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
sc = pyspark.SparkContext()
sql_sc = pyspark.sql.SQLContext(sc)

In [ ]:
s_df = sql_sc.createDataFrame(X)

In [ ]:
#s_df.values()
print(type(s_df))
print(type(s_df.rdd))

s_df.rdd.first()

In [ ]:
# Load and parse the data
#data = sc.textFile("data/mllib/als/test.data")
#ratings = data.map(lambda l: l.split(','))\
#    .map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))

# Build the recommendation model using Alternating Least Squares
rank = 10
numIterations = 10
model = ALS.train(s_df.rdd, rank, numIterations, 0.1)

# Evaluate the model on training data
testdata = s_df.rdd.map(lambda p: (p[0], p[1]))
predictions = model.predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))
ratesAndPreds = s_df.rdd.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("Mean Squared Error = " + str(MSE))

# Save and load model
#model.save(sc, "target/tmp/myCollaborativeFilter")
#sameModel = MatrixFactorizationModel.load(sc, "target/tmp/myCollaborativeFilter")

In [ ]:
print(predictions.first())

In [ ]:
testDataRaw = pd.read_csv('sampleSubmission.csv')

#testData['User'] = [(ID.split('_')[0])[1:] for ID in testData['Id']]
#testData['Movie'] = [(ID.split('_')[1])[1:] for ID in testData['Id']]

XTest = testDataRaw[['User', 'Movie']]
test = sql.createDataFrame(XTest)

In [ ]:
print(test.rdd.first())

In [ ]:
predictionsTest = model.predictAll(test.rdd).map(lambda r: (r[0], r[1], r[2]))
#ratesAndPreds = s_df.rdd.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
#MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
#print("Mean Squared Error = " + str(MSE))

In [ ]:
print(predictionsTest.first())

In [ ]:
df = predictionsTest.toDF()
predictionsDF = df.toPandas()
predictionsDF.head()

In [ ]:
predictionsDF.columns = ['User', 'Movie', 'Prediction']
predictionsDF.head()

In [ ]:
predictionsDF.sort_values(['Movie','User'], ascending=[1,1], kind='mergesort', inplace=True)
predictionsDF.head()

In [ ]:
predictionsDF.Prediction = predictionsDF.Prediction.round()
predictionsDF.head()

In [ ]:
#testData = pd.read_csv('sampleSubmission.csv')

In [ ]:
predictionsDF.reset_index(drop=True,inplace=True)
predictionsDF.head()

In [ ]:
testDataRaw.head()
testDataRaw['Prediction'] = predictionsDF['Prediction']
testDataRaw.to_csv('submission.csv', index = False)

In [ ]:
testDataRaw.head()